In [37]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import preprocess

res_path = "./macroData/"

macro = 100

noiseLevel = "low"
modelName = "RNN"
plot = False

In [38]:
outerScenarios = np.load(f"{res_path}outerScenarios.npy")
trueLosses = np.load(f"{res_path}trueLosses.npy")
 
seed = 22
np.random.seed(seed)

mse_trains = []
mse_tests = []
mse_trues = []

for m in range(macro):

    try:
        seed_m = np.random.randint(0, 32767)

        loss_train = np.load(f"{res_path}loss_{noiseLevel}Noise_{m}.npy")

        loss_mean = np.mean(loss_train)
        loss_std = np.std(loss_train)

        loss_hat = np.load(f"{res_path}loss_{noiseLevel}Noise_{modelName}_{m}.npy")

        y_hat = (loss_hat - loss_mean) / loss_std
        y_train = (loss_train - loss_mean) / loss_std
        y_true = (trueLosses - loss_mean) / loss_std

        _, _, y_train, y_test = train_test_split(outerScenarios, y_train, test_size=3000, random_state=seed_m)
        _, _, y_hat_train, y_hat_test = train_test_split(outerScenarios, y_hat, test_size=3000, random_state=seed_m)

        mse_train = np.mean((y_hat_train.flatten() - y_train.flatten()) ** 2)
        mse_test = np.mean((y_hat_test.flatten() - y_test.flatten()) ** 2)
        mse_true = np.mean((y_hat.flatten() - y_true.flatten()) ** 2)

        mse_trains.append(mse_train)
        mse_tests.append(mse_test)
        mse_trues.append(mse_true)

        print(f"Macro {m}: MSE Train: {mse_train}, MSE Test: {mse_test}, MSE True: {mse_true}")

    except:
        print(f"Result for macro {m + 1} does not exist.")
        continue

print(f"MSE Train: {np.nanmean(mse_train)}, MSE Test: {np.nanmean(mse_test)}, MSE True: {np.nanmean(mse_true)}")

Macro 0: MSE Train: 0.08181265190722337, MSE Test: 0.09673996134085779, MSE True: 0.07126375929948074
Macro 1: MSE Train: 0.06683164647770216, MSE Test: 0.07261154176712792, MSE True: 0.05545627245953775
Macro 2: MSE Train: 0.08005255157016032, MSE Test: 0.08297561133596586, MSE True: 0.06768218583055795
Macro 3: MSE Train: nan, MSE Test: nan, MSE True: nan
Macro 4: MSE Train: 0.08397779548893798, MSE Test: 0.08661632771046135, MSE True: 0.07141565507392877
Macro 5: MSE Train: 0.07445168654449023, MSE Test: 0.08255583960488115, MSE True: 0.06281322225318403
Macro 6: MSE Train: 0.07640832331432194, MSE Test: 0.07713138460577128, MSE True: 0.06392521400665974
Macro 7: MSE Train: 0.06890917089540964, MSE Test: 0.07896723793162982, MSE True: 0.05820081807024959
Macro 8: MSE Train: 0.06790016445460584, MSE Test: 0.07687641325251059, MSE True: 0.05653748836890503
Macro 9: MSE Train: 0.07977485783216161, MSE Test: 0.09635402958893866, MSE True: 0.06934615873388608
Macro 10: MSE Train: 0.07074

In [13]:
import numpy as np
import os

res_path = "./macroData/"

outerScenarios = np.load(f"{res_path}outerScenarios.npy")
print(outerScenarios.shape)

# Separate into several separate npy file with shape (10090, 241)
for i in range(10):
    np.save(f"{res_path}outerScenarios_{i}.npy", outerScenarios[i*10090:(i+1)*10090, :])

# Read them back
outerScenariosNew = []
for i in range(10):
    outerScenario = np.load(f"{res_path}outerScenarios_{i}.npy")
    outerScenariosNew.append(outerScenario)

outerScenariosNew = np.concatenate(outerScenariosNew, axis=0)

print(outerScenariosNew.shape)

# Check if they are the same
print(np.all(outerScenarios == outerScenariosNew))

# if they are the same, delete the old files
if np.all(outerScenarios == outerScenariosNew):
    os.remove(f"{res_path}outerScenarios.npy")



(100900, 241)
(100900, 241)
True


In [ ]:
def load_outerScenarios(res_path):
    outerScenarios = []
    for i in range(10):
        outerScenario = np.load(f"{res_path}outerScenarios_{i}.npy")
        outerScenarios.append(outerScenario)
    return np.concatenate(outerScenarios, axis=0)